# 使用 Meta 家族模型建構專案

## 介紹

本課程將涵蓋：

- 探討 Meta 家族的兩大主力模型——Llama 3.1 和 Llama 3.2
- 了解每個模型的適用情境與案例
- 透過程式碼範例展示各模型的獨特功能

## Meta 家族模型

在本課程中，我們將深入了解 Meta 家族（又稱 "Llama Herd"）的兩款模型——Llama 3.1 和 Llama 3.2

這些模型有多種版本，並可在 Github Model 市集取得。想了解更多如何利用 Github Models [原型設計 AI 模型](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst)，可參考官方說明。

模型版本：
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*注意：Llama 3 也可在 Github Models 上取得，但本課程不會介紹該模型*


## Llama 3.1

擁有 4050 億參數的 Llama 3.1 屬於開源 LLM 的一員。

這個模型是對先前版本 Llama 3 的升級，帶來了：

- 更大的上下文視窗 —— 128k tokens 對比 8k tokens
- 更高的最大輸出 tokens —— 4096 對比 2048
- 更佳的多語言支援 —— 由於訓練 tokens 數量增加

這些提升讓 Llama 3.1 能夠在開發 GenAI 應用時，處理更複雜的情境，包括：
- 原生函式呼叫 —— 能夠在 LLM 工作流程之外呼叫外部工具和函式
- 更好的 RAG 效能 —— 受益於更大的上下文視窗
- 合成資料生成 —— 能夠為微調等任務產生有效的資料


### 原生函式呼叫

Llama 3.1 已經經過微調，能更有效地進行函式或工具呼叫。它也內建了兩個工具，模型可以根據使用者的提示自動判斷是否需要使用這些工具。這兩個工具分別是：

- **Brave Search** - 可以透過網路搜尋獲取最新資訊，例如天氣狀況
- **Wolfram Alpha** - 可以用來進行更複雜的數學運算，因此不需要自己撰寫相關函式

你也可以自訂自己的工具，讓 LLM 進行呼叫。

在下方的程式碼範例中：

- 我們在系統提示中定義了可用的工具（brave_search, wolfram_alpha）。
- 傳送一個詢問特定城市天氣的使用者提示。
- LLM 會回應一個對 Brave Search 工具的呼叫，格式如下：`<|python_tag|>brave_search.call(query="Stockholm weather")`

*注意：這個範例只會執行工具呼叫，如果你想取得查詢結果，需要先到 Brave API 頁面註冊免費帳號，並自行定義該函式*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

雖然 Llama 3.1 是一個大型語言模型，但它有一個限制，就是缺乏多模態能力。也就是說，無法同時處理像圖片這類不同型態的輸入並給出回應。這項能力正是 Llama 3.2 的主要特色之一。這些新功能還包括：

- 多模態支援 - 能夠同時處理文字和圖片提示
- 小型到中型模型變體（11B 和 90B）- 提供彈性的部署選擇
- 僅限文字的變體（1B 和 3B）- 讓模型可以部署在邊緣或行動裝置上，並帶來低延遲

多模態支援代表開源模型領域的一大進展。下面的程式碼範例同時輸入圖片和文字提示，讓 Llama 3.2 90B 進行圖片分析。

### Llama 3.2 的多模態支援


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## 學習不止於此，繼續你的旅程

完成本課程後，歡迎前往我們的 [生成式 AI 學習專區](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst)，持續提升你對生成式 AI 的認識！



---

**免責聲明**：  
本文件係使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻譯。雖然我們力求準確，但請注意，自動翻譯可能包含錯誤或不精確之處。原始語言版本應視為具權威性的來源。對於重要資訊，建議尋求專業人工翻譯。我們對因使用本翻譯而產生的任何誤解或誤釋不承擔任何責任。
